In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from seirt_mesa import SEIRTModel
from cpyment import CModel

In [6]:
N = 4000
I0 = 0.01
beta = 0.4
gamma = 0.1
delta = 0.1
theta = 0.1
kappa = 0.03
eta = 1

m = SEIRTModel(N, I0=I0, beta=beta, gamma=gamma, delta=delta, theta=theta, kappa=kappa, 
               eta = 0, dt=0.05)

In [7]:
# Compartment version
cm = CModel('SEIRT')

cm.set_coupling_rate('S*I:S=>E', beta/N)
cm.set_coupling_rate('E:E=>I', gamma)
cm.set_coupling_rate('E:E=>T', eta*theta)
cm.set_coupling_rate('I:I=>R', delta)
cm.set_coupling_rate('I:I=>T', theta*(1+eta))
cm.set_coupling_rate('T:T=>R', kappa)

cmNoCT = CModel('SEIRT')

cmNoCT.set_coupling_rate('S*I:S=>E', beta/N)
cmNoCT.set_coupling_rate('E:E=>I', gamma)
cmNoCT.set_coupling_rate('I:I=>R', delta)
cmNoCT.set_coupling_rate('I:I=>T', theta)
cmNoCT.set_coupling_rate('T:T=>R', kappa)

In [ ]:
steps = 2000
for i in range(steps):
    m.step()

In [ ]:
t = m.dt*np.arange(steps)
y0 = np.array([N*(1-I0), 0, N*I0, 0, 0])

traj = cm.integrate(t, y0)
trajNoCT = cmNoCT.integrate(t, y0)

In [ ]:
df = m.datacollector.get_model_vars_dataframe()

In [ ]:
fig, ax = plt.subplots()

colors = [
    (0.0, 0.5, 1.0),
    (1.0, 0.6, 0.0),
    (1.0, 0.2, 0.1),
    (0.0, 0.7, 0.4),
    (0.8, 0.0, 0.9)
]

ax.set_ylim(0, 500)
for i in range(5):
    ax.plot(t, traj['y'][:,i], c=colors[i], label='SEIRT'[i], lw=2.0)
    ax.plot(t, trajNoCT['y'][:,i], '--', c=colors[i], lw=2.0)
    ax.plot(t, df.values[:,i+1], c=colors[i], lw=0.6)
    
ax.legend()